Ziel dieses Programmes ist es, die vom zentralen System angelegten Daten mit real Existierenden in verbindung zu bringen. Idealerweise sollten alle Funktionen für ein Android-System kompatibel sein. <br>
Es gibt 3.5 Möglichkeiten die Objekte zu finden. Mit Optical-Character-Recognition die Id auf den Produkt zu erkennen, QR-Codes aus Namen und Id zu erkennen und das Objekt manuell im System zu finden. Letzteres kann durch Durchklicken von Klassen oder durch Suchen von Schlüsselworten geschehen. <br>
Anschließend verläuft das System wie bei einer Shopping-Webseite, um vorhandenes Verhalten der Nutzer auszunutzen.

scaling

remove überflüssige layouts -> WA_DeleteOnClose muss noch geset werden? / write terminus_abolutus

beleg

sumup api

readnumbers/bessere identifizierung (liegt vielleicht an eigener Webcam)

In [1]:
import sys 
from PyQt6.QtWidgets import *
import os.path
import pandas as pd
from PyQt6.QtCore import Qt, QTimer, QDateTime
import qrcode
import time
from functools import partial
from PyQt6 import QtTest
import pickle
from PyQt6.QtGui import QPixmap
from PyQt6.QtGui import *
import datetime
import cv2
from pyzbar.pyzbar import decode
import cryptpandas as crp
from PyQt6.QtCore import pyqtSignal
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [2]:
""" #readnr is supposed to read the product nr/ id via a camera. However even with heavy hitters like pytesseract, it will not allow me 
#to properly read the tests. This can be because of my bad webcam or because I expect to much from external sources
#however training my own OCR requires training data which not only includes numers but letters and worse special signs
if not os.path.isfile("processorpkl"):
    from transformers import TrOCRProcessor, VisionEncoderDecoderModel
    processor=TrOCRProcessor.from_pretrained("microsoft/trocr-large-printed")
    model=VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-large-printed")
    with open("modelpkl", 'rb') as file:  
        pickle.dump(model, file)
    with open("processorpkl", 'rb') as file:  
        pickle.dump(processor, file)
        
else:
    dbfile = open('modelpkl', 'rb')    
    model=pickle.load(dbfile)
    dbfile.close()
    dbfile = open('processorpkl', 'rb')    
    processor=pickle.load(dbfile)
    dbfile.close()
"""

' #readnr is supposed to read the product nr/ id via a camera. However even with heavy hitters like pytesseract, it will not allow me \n#to properly read the tests. This can be because of my bad webcam or because I expect to much from external sources\n#however training my own OCR requires training data which not only includes numers but letters and worse special signs\nif not os.path.isfile("processorpkl"):\n    from transformers import TrOCRProcessor, VisionEncoderDecoderModel\n    processor=TrOCRProcessor.from_pretrained("microsoft/trocr-large-printed")\n    model=VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-large-printed")\n    with open("modelpkl", \'rb\') as file:  \n        pickle.dump(model, file)\n    with open("processorpkl", \'rb\') as file:  \n        pickle.dump(processor, file)\n        \nelse:\n    dbfile = open(\'modelpkl\', \'rb\')    \n    model=pickle.load(dbfile)\n    dbfile.close()\n    dbfile = open(\'processorpkl\', \'rb\')    \n    processor=pickle

In [3]:
Wahrung="€"
df=[]
sales=pd.DataFrame()
l_Event=[]

In [4]:
def read_files():
    global df
    if len(df)!=0:
        df=[]
    for i in ["objects.json", "hierachies.json"]:
        if os.path.isfile(i):
            df.append(pd.read_json(i))
        else:
            df.append(pd.DataFrame())
    df[0]=df[0][~(df[0]["name"].str.startswith('!', na=True))]
    df[0]=df[0].loc[df[0]["id"]!="#00000"]
    df[0]=df[0][~(df[0].name.str.startswith("!", na=False))]
    if (len(df[0])>0) and (len(df[0].columns)!=8):
        return "CorruptedFile"
    return False

In [5]:
def getpassw():
    #from cryptography.fernet import Fernet
    global df
    if os.path.isfile("keypickle"):
        dbfile = open('keypickle', 'rb')    
        key=pickle.load(dbfile)
        dbfile.close()
    else:
        key = Fernet.generate_key()
        dbfile=open('keypickle', 'ab')
        pickle.dump(key, dbfile)
        dbfile.close()
    return key


In [6]:
def safe_verkauf():
    global sales
    x=sales["id"].apply(hash)
    y=sales["time"].apply(hash)
    z=sales["total"].apply(hash)
    hashv=hash(x.sum()+y.sum()+z.sum())
    pw=getpassw()    
    crp.to_encrypted(sales, password=str(pw), path='sales.crypt')
    dbfile=open('hashpickle', 'ab')
    pickle.dump(hashv, dbfile)
    dbfile.close()

In [7]:
def read_verkauf():
    global sales
    if not (os.path.isfile("sales.crypt") and os.path.isfile("hashpickle")):
        sales=pd.DataFrame()
        return True
    pw=getpassw()    
    sales=crp.read_encrypted(path='sales.crypt', password=str(pw))
    dbfile=open('hashpickle', 'rb')    
    hashv=pickle.load(dbfile)
    dbfile.close()
    x=sales["id"].apply(hash)
    y=sales["time"].apply(hash)
    z=sales["total"].apply(hash)
    hashv2=hash(x.sum()+y.sum()+z.sum())
    if hashv!=hashv:
        return "CorruptedFile"
        print("corrupt")
    return False

In [8]:
def read_settings():
    global l_Event
    if not os.path.isfile("settings"): 
        return True
    dbfile=open('settings', 'rb')    
    settings=pickle.load(dbfile)
    ph=next(iter(test))
    if ph>datetime.datetime.now()-datetime.timedelta(days=1):
        l_Event=settings[ph]
    dbfile.close()

In [9]:
def safe_settings():
    global l_Event
    dbfile=open('settings', 'ab')
    settings={datetime.datetime.now():l_Event}
    pickle.dump(settings, dbfile)
    dbfile.close()

In [10]:
def can_float(i):
    try:
        float(i)
        return True
    except:
        return False

In [11]:
def del_widget(widget, layout=None): #no table but used often
    if layout:
        layout.removeWidget(widget)
    widget.deleteLater()
    widget=None

In [12]:
read_files()
read_verkauf()
wagen=pd.DataFrame(columns=(df[0].columns.append(pd.Index(["amount"]))))
app = QApplication([])
enviroment=QApplication(sys.argv)
screen=app.primaryScreen()
Grosse=screen.size()
print(Grosse)

PyQt6.QtCore.QSize(1536, 864)


In [13]:
class LayoutWidget(QWidget): #shows an item #we could heve presented it just as before in a table but we pursued the shopping 
    clicked = pyqtSignal() #card so intesly that we can just copy the presentation to. Even when we make this harde on ourselfs.
    #self?
    def __init__(self, parent=None, pds=pd.DataFrame()):
        super().__init__(parent)
        self.id=[pds["id"]]
        self.item=QHBoxLayout(self)

        if str(pds["image"])!="": #need?
            itemimage=QPixmap(str(pds["image"]))
            itemimage=itemimage.scaledToWidth(70)#int(Grosse.width()/2))
            l1=QLabel()
            l1.setPixmap(itemimage)
            self.item.addWidget(l1)

        descr=QVBoxLayout()
        l2=QLabel(str(pds["name"]))
        l2.setFont(QFont('Arial', 15))
        descr.addWidget(l2)

        l1=QLabel("id: "+str(pds["id"]))
        l1.setFont(QFont('Arial', 8))
        descr.addWidget(l1, alignment=Qt.AlignmentFlag.AlignRight)        

        names=QHBoxLayout()
        for i in pds["altname"]:
            l1=QLabel(str(i))
            l1.setFont(QFont('Arial', 12))
            names.addWidget(l1)
        descr.addLayout(names)

            
        l1=QLabel(str(pds["price"])+Wahrung)
        l1.setFont(QFont('Arial', 15))
        descr.addWidget(l1, alignment=Qt.AlignmentFlag.AlignLeft)
            
        name=QHBoxLayout()
        for i in pds["tag"]:
            l1=QLabel(str(i))
            l1.setFont(QFont('Arial', 10))
            name.addWidget(l1)
        descr.addLayout(name)
        descr.setSpacing(0)
        self.item.setSpacing(0)
        self.item.addLayout(descr)
        self.item.addStretch()

    def mouseReleaseEvent(self, event): #extra class because when only one part of object is clicked, make something
        super().mouseReleaseEvent(event)
        self.clicked.emit()

In [14]:
def calcChange(x): #calculate exchange money
    global Wahrung
    notes=[50,20,10,5,2,1,0.5,0.2,0.1,0.05,0.02,0.01]
    changes={}
    result=""

    for i in notes:
        changes[i]=0
        while x-i>=-0.001:###################rundungsfehler pythons
            changes[i]+=1
            x-=i
            
    for note, amount in changes.items():
        if amount>0:
            result+=str(note)+Wahrung+ " "+str(amount)+"\n"
    return result    

In [15]:
class Hierachie(QWidget): #same as in central system: gets children from parent but here also objects belonging to a hierachie
    
    def __init__(self, father=None, parent=None):
        super(Hierachie, self).__init__(parent)
        global df
        self.widget={}
        self.layout=QVBoxLayout()
        self._delete(father)
        self._buildChild(father)
        self._buildItems(father)
        

    def _delete(self, father):
        self.b_back=QPushButton("<-")
        
        self.b_back.setFixedSize(20, 20)
        if father!=[]:
            self.layout.addWidget(self.b_back, alignment=Qt.AlignmentFlag.AlignLeft)
        
    def _buildChild(self, father):
        global df
        self.child={}
        
        if df[1].empty:
            return
        if father==[]:
            ph=df[1][df[1].parents.str.len()==0]#["children"].iloc[0]
            for i, entry in ph.iterrows():
                self.child[i]=QPushButton(str(ph.name[i])) 
                self.layout.addWidget(self.child[i])
        else:
            ph=df[1]["children"][father[1]]
            #    ph=df[1][df[1]["parents"].apply(lambda x: father in x)]["children"].index[0] child 
            for i in ph:
                self.child[i[1]]=QPushButton(str(i[0])) 
                self.layout.addWidget(self.child[i[1]])
                
    def _buildItems(self, father): #scroll
        global df
        if father==[]:
            return
        ergebnis=df[0][df[0].hierachie.apply(lambda x: x==father)]
        for index, pds in ergebnis.iterrows():
            self.widget[pds["id"]]=LayoutWidget(self, pds)
            self.layout.addWidget(self.widget[pds["id"]])

        #Frame.show()

In [16]:
class App_Hierachie(QWidget): #makes click though of hierachies, but no adding instead shows objects atributed to class
    
    def __init__(self, parent=None, flag=False):
        global df
        super(App_Hierachie, self).__init__(parent)
        self.layout=QVBoxLayout()
        self.setLayout(self.layout)
        
        self.bM=QPushButton()
        self._back()
        self.lerrors=QLabel("")

        self.you=QLabel("")
        self.layout.addWidget(self.you, alignment=Qt.AlignmentFlag.AlignHCenter)
        self._showHierachie(parents=[],flag=False)

    #def _createScroll(self): needed for larger inputs?

    def _showHierachie(self, parents, flag=True):
        global df
        if flag:
            for id, w in self.hierachie.widget.items():
                del_widget(w, self.hierachie.layout)
            for id, w in self.hierachie.child.items():
                del_widget(w, self.hierachie.layout)
            self.hierachie.deleteLater()
                 
        self.hierachie=Hierachie(parents)
        #self.hierachie.b_delete.clicked.connect(self._remove)
        #self.hierachie.b_deleteC.clicked.connect(self._removeAll)
        
        if parents!=[]:
            
            self.you.setText(parents[0])
            dfx=df[1].dropna() #################################################inefficient
            ph=dfx[dfx.children.apply(lambda x: parents in x)]
            if len(ph)==0:
                self.hierachie.b_back.clicked.connect(lambda:self._showHierachie(parents=[],flag=True))
            else:
                ph=ph.index[0]
                self.hierachie.b_back.clicked.connect(partial(self._showHierachie,[df[1]["name"][ph],ph],flag=True))
        else:
            self.you.setText("")


        for index, b in self.hierachie.child.items():
            b.clicked.connect(partial(self._showHierachie,[df[1]["name"][index],index],flag=True)) #partial need flag=True
            
        if flag:
            del_widget(self.frame, self.layout) #############dirty but update doesn't work
            self.frame=QFrame()
            self.frame.setLayout(self.hierachie.layout)
            self.layout.addWidget(self.frame)
            self.frame.show()

        else:
            self.frame=QFrame()
            self.frame.setLayout(self.hierachie.layout)
            self.layout.addWidget(self.frame)
            self.frame.show()

    def _back(self):
        self.b_back=QPushButton("<-")
        self.b_back.setFixedSize(20, 20)
        self.layout.addWidget(self.b_back, alignment=Qt.AlignmentFlag.AlignLeft)
        
    def showError(self, msg):
        self.lerrors.setText(msg)
        QtTest.QTest.qWait(5000)
        self.lerrors.setText("")

In [17]:
class App_search(QWidget): #finds objects that contains tosearch in their name, altname or tag #implement prorities?
    
    def __init__(self, tosearch, parent=None):
        super(App_search, self).__init__(parent)
        global df

        self.layout=QVBoxLayout()
        self.bM=QPushButton()
        self.setLayout(self.layout)
        self._back()
        self._search(tosearch)

    def _back(self):
        self.b_back=QPushButton("<-")
        self.b_back.setFixedSize(20, 20)
        self.layout.addWidget(self.b_back, alignment=Qt.AlignmentFlag.AlignLeft)
        

    def _createScroll(self):
        self.scroll=QScrollArea()
        self.scroll.setVerticalScrollBarPolicy(Qt.ScrollBarPolicy.ScrollBarAlwaysOn) #labels get cut off otherwise
        self.scroll.setHorizontalScrollBarPolicy(Qt.ScrollBarPolicy.ScrollBarAlwaysOff)
        self.scroll.setWidgetResizable(True)
        self.scroll.setWidget(self.item)
        self.scroll.setMaximumHeight(200)
        self.layout.addWidget(self.scroll)
    
    def _search(self, tosearch):
        tosearch=str(tosearch).lower()
        global df
        
        q1=df[0]["name"].apply(lambda x: tosearch in x.lower())
        #q1=df[0]["name"].astype(str).str.contains(tosearch, regex=False)
        q2=df[0]["tag"].apply(lambda x: any(tosearch in y.lower() for y in x))
        q3=df[0]["altname"].apply(lambda x: any(tosearch in y.lower() for y in x))
        self.ergebnis=df[0][q1|q2|q3].reset_index(drop=True)

        self.item=QWidget()
        self.widget={}
        self.items=QVBoxLayout()
        for index, pds in self.ergebnis.iterrows():
            self.widget[pds["id"]]=LayoutWidget(self, pds)
            self.items.addWidget(self.widget[pds["id"]])####somehow the widgets get suished if they are too much 
        self.item.setLayout(self.items)
        self._createScroll() 

In [18]:
class App_rueckgeld(QWidget): #showing the exchange money
    
    def __init__(self, amount, parent=None):
        super(App_rueckgeld, self).__init__(parent)
        global df

        self.layout=QVBoxLayout()
        self.setLayout(self.layout)
        self.bM=QPushButton()
        self._back()
        self._finish(amount)

    def _back(self):
        self.b_back=QPushButton("<-")
        self.b_back.setFixedSize(20, 20)
        self.layout.addWidget(self.b_back, alignment=Qt.AlignmentFlag.AlignLeft)
        
        
    def _finish(self, amount):
        global Wahrung
        search=QHBoxLayout()
        self.f_bekommen=QLineEdit()
        self.f_bekommen.setPlaceholderText("bekommen")
        self.ruckgeld=QLabel("")
        self.b_finish=QPushButton("Finish")
        self.layout.addWidget(QLabel(str(amount)+Wahrung))
        self.layout.addWidget(self.f_bekommen)
        self.layout.addWidget(self.ruckgeld)
        self.layout.addWidget(self.b_finish)
        self.f_bekommen.textEdited.connect(lambda:(self._setchange(amount)))

    def _setchange(self, amount):
        rein=self.f_bekommen.text()
        if not can_float(rein):
            self.ruckgeld.setText("Bitte nur Nummern eingeben")
            return
        rein=float(rein)
        if rein<amount:
            self.ruckgeld.setText("zu wenig")
            return
        self.ruckgeld.setText(str(round(rein-amount,2))+"\n"+calcChange(rein-amount))

In [19]:
class App_korb(QWidget): #shopping cart
    
    def __init__(self, parent=None):
        super(App_korb, self).__init__(parent)
        global df

        self.layout=QVBoxLayout()
        #centralWidget=QWidget(self)
        self.setLayout(self.layout)
        #self.setCentralWidget(centralWidget)
        self._back()
        self._showItem()
        self._createScroll()

    def _back(self):
        self.b_back=QPushButton("<-")
        self.b_back.setFixedSize(20, 20)
        self.layout.addWidget(self.b_back, alignment=Qt.AlignmentFlag.AlignLeft)

    def _createScroll(self):
        self.scroll=QScrollArea()
        self.scroll.setLayout(self.items)
        self.scroll.setWidgetResizable(True)
        self.scroll.setMaximumHeight(200)
        self.layout.addWidget(self.scroll)

    def _showItem(self): #beim zufügen von wagen duplikate verdoppen
        global df
        global wagen
        global Grosse
        global Wahrung

        self.items=QVBoxLayout()
        self.item={}
        self.b_delI={}
        self.desc={}
        
        
        for index, pds in wagen.iterrows():
            self.item[pds["id"]]=QHBoxLayout()
            if pds["image"]!="":
                itemimage=QPixmap(pds["image"])
                itemimage=itemimage.scaledToWidth(70)#int(Grosse.width()/2))
                self.label = QLabel()
                self.label.setPixmap(itemimage)
                self.item[pds["id"]].addWidget(self.label)

            self.desc[pds["id"]]=QVBoxLayout()
            l1=QLabel(str(pds["name"]))
            l1.setFont(QFont('Arial', 15))
            self.desc[pds["id"]].addWidget(l1)

            l1=QLabel("id: "+str(pds["id"]))
            l1.setFont(QFont('Arial', 8))
            self.desc[pds["id"]].addWidget(l1, alignment=Qt.AlignmentFlag.AlignRight)        

            names=QHBoxLayout()
            for i in pds["altname"]:
                l1=QLabel(str(i))
                l1.setFont(QFont('Arial', 12))
                self.names.addWidget(l1)
            self.desc[pds["id"]].addLayout(names)

            nbers=QHBoxLayout()
            l1=QLabel(str(pds["price"])+Wahrung)
            l1.setFont(QFont('Arial', 15))
            self.l_amt=QLabel(str(pds["amount"])+" Stk.="+str(pds["amount"]*pds["price"]))###############################
            self.l_amt.setFont(QFont('Arial', 13))
            nbers.addWidget(l1)
            nbers.addWidget(self.l_amt)
            self.desc[pds["id"]].addLayout(nbers)
            
            names=QHBoxLayout()
            for i in pds["tag"]:
                l1=QLabel(str(i))
                l1.setFont(QFont('Arial', 10))
                self.names.addWidget(l1)
            self.desc[pds["id"]].addLayout(names)
            self.item[pds["id"]].addLayout(self.desc[pds["id"]])
            self.b_delI[pds["id"]]=QPushButton("x")
            self.item[pds["id"]].addWidget(self.b_delI[pds["id"]])
            
            self.items.addLayout(self.item[pds["id"]])
            
        price=QHBoxLayout()
        self.endsum=sum(wagen.amount*wagen.price)
        if (self.endsum) or (self.endsum==0):
            self.l_endsum=(QLabel(str(self.endsum)+Wahrung))
            self.l_endsum.setFont(QFont('Arial', 17))
            price.addWidget(self.l_endsum)
            
        self.f_rabatt=QLineEdit() 
        self.f_rabatt.setPlaceholderText("Rabatte e.g. -20% or =9")
        price.addWidget(self.f_rabatt)
        self.layout.addLayout(price)
        self.b_abgabe=QPushButton("Abgabe") 
        self.layout.addWidget(self.b_abgabe)


In [20]:
class App_item(QWidget): #displays item before shopping cart
    
    def __init__(self, pds, parent=None):
        super(App_item, self).__init__(parent)
        global df
        global Wahrung
        self.bM=QPushButton()
        self.pds=pds#.iloc[0]bad for wagen
        self.p=pds.index[0]
        self.item=QVBoxLayout()
        self.setLayout(self.item)
        self._back()
        self._showItem()

    def _back(self):
        self.b_back=QPushButton("<-")
        self.b_back.setFixedSize(20, 20)
        self.item.addWidget(self.b_back, alignment=Qt.AlignmentFlag.AlignLeft)

    def _showItem(self):
        global df
        global Wahrung
        
        l1=QLabel(str(self.pds["name"][self.p]))
        l1.setFont(QFont('Arial', 15))
        self.item.addWidget(l1)

        l1=QLabel("id: "+str(self.pds["id"][self.p]))
        l1.setFont(QFont('Arial', 8))
        self.item.addWidget(l1, alignment=Qt.AlignmentFlag.AlignRight)        

        names=QHBoxLayout()
        for i in self.pds["altname"][self.p]:
            l1=QLabel(str(i))
            l1.setFont(QFont('Arial', 12))
            names.addWidget(l1)
        self.item.addLayout(names)

        l1=QLabel(str(self.pds["price"][self.p])+Wahrung)
        l1.setFont(QFont('Arial', 15))
        self.item.addWidget(l1)  

        names=QHBoxLayout()
        for i in self.pds["tag"][self.p]:
            l1=QLabel(str(i))
            l1.setFont(QFont('Arial', 10))
            names.addWidget(l1)
        self.item.addLayout(names)
        
        self.b_submit=QPushButton("In den Wagen")
        self.item.addWidget(self.b_submit)


In [21]:
class App_settings(QWidget): #allows to set certain tags in attribution to the event
    
    def __init__(self, parent=None):
        super(App_settings, self).__init__(parent)
        global df
        global l_Event
        
        self.layout1=QVBoxLayout()
        self.setLayout(self.layout1)
        self._back()
        self.bM=QPushButton()
        self._createHeader()

    def _back(self):
        self.b_back=QPushButton("<-")
        self.b_back.setFixedSize(20, 20)
        self.layout1.addWidget(self.b_back, alignment=Qt.AlignmentFlag.AlignLeft)
        
    def _createHeader(self):
        self.r_Event={}#remove event
        self.q_Event={} #display event
        self.dt=QLabel()
        self.layout1.addWidget(self.dt)
        self.f_Event=QLineEdit() #form event
        
        self.timer=QTimer(self)
        self.timer.timeout.connect(self._giveTime)
        self.timer.start(1000) 
        
        self.f_Event.setPlaceholderText("Besonderes Event")
        self.layout1.addWidget(self.f_Event)
        
        self.b_add=QPushButton("add")
        self.layout1.addWidget(self.b_add)

        self.lerrors=QLabel()
        self.layout1.addWidget(self.lerrors)
        
        self.events=QVBoxLayout()
        self.layout1.addLayout(self.events)
        self.b_add.clicked.connect(self._addEvent)

    def _addEvent(self):
        global l_Event
        if self.f_Event.text() in l_Event:
            self.showError("Event existiert bereits")
        l_Event.append(self.f_Event.text())
        event=QHBoxLayout()
        self.q_Event[self.f_Event.text()]=QLabel(self.f_Event.text())
        event.addWidget(self.q_Event[self.f_Event.text()])
        self.r_Event[self.f_Event.text()]=QPushButton("X")
        
        event.addWidget(self.r_Event[self.f_Event.text()])
        self.events.addLayout(event)
        
        self.r_Event[self.f_Event.text()].clicked.connect(partial(self._remove, self.f_Event.text()))
        safe_settings()

    def _giveTime(self):
        currentdate=QDateTime.currentDateTime()
        self.dt.setText(currentdate.toString('dd-MM-yyyy hh:mm:ss'))

    def showError(self, msg):
        self.lerrors.setText(msg)
        QtTest.QTest.qWait(5000)
        self.lerrors.setText("")

    def _remove(self, id):
        global l_Event
        self.r_Event[id].deleteLater()
        self.q_Event[id].deleteLater()
        safe_settings()
        l_Event.remove(id)
        

In [22]:
class Control_rueck:
    def __init__(self, view, master):
        self._view = view
        self._master=master
        self._connectSignalsAndSlots()
        
    def _connectSignalsAndSlots(self):
        self._view.bM.clicked.connect(lambda:(self._buildExpression("")))######################Dont ask why
        self._view.b_finish.clicked.connect(self._back)
        self._view.b_back.clicked.connect(self._back)

    def _buildExpression(self,x):
        return

    def _back(self):
        print("aaaaaaaa")
        self._master.central_widget.setCurrentWidget(self._master.main_screen)#also found at .widget(0)

In [23]:
class Control_item:
    def __init__(self, view, master):
        self._view = view
        self._master=master
        self._connectSignalsAndSlots()

    def _buildExpression(self, x):
        return
        
    def _connectSignalsAndSlots(self):
        self._view.bM.clicked.connect(lambda:(self._buildExpression("")))######################Dont ask why
        self._view.b_back.clicked.connect(self._back)
        self._view.b_submit.clicked.connect(self._submit)

    def _back(self):
        print("aaaaaaaa")
        self._master.central_widget.setCurrentWidget(self._master.main_screen)#also found at .widget(0)
        
    def _submit(self):
        global wagen
        if wagen[wagen["id"]==self._view.pds["id"][self._view.p]].empty:
            wagen=pd.concat([wagen, self._view.pds])
            wagen.loc[wagen["id"]==self._view.pds["id"][self._view.p], "amount"]=1
        else:
            wagen.loc[wagen["id"]==self._view.pds["id"][self._view.p], "amount"]+=1

        print(wagen)
            
        self.korb=App_korb(self._master)
        Control_korb(view=self.korb, master=self._master)

        self._master.central_widget.addWidget(self.korb)
        self._master.central_widget.setCurrentWidget(self.korb)

In [24]:
class Control_settings:
    def __init__(self, view, master):
        self._view = view
        self._master=master
        self._connectSignalsAndSlots()
        
    def _connectSignalsAndSlots(self):
        global df
        self._view.bM.clicked.connect(lambda:(self._buildExpression("")))######################Dont ask why
        self._view.b_back.clicked.connect(self._back)

    def _buildExpression(self,x):
        return

    def _back(self):
        self._master.central_widget.setCurrentWidget(self._master.main_screen)#also found at .widget(0)

In [25]:
class Control_search:
    def __init__(self, view, master):
        self._view = view
        self._master=master
        self._connectSignalsAndSlots()

    def _buildExpression(self, x):
        return
        
    def _connectSignalsAndSlots(self):
        global df
        self._view.bM.clicked.connect(lambda:(self._buildExpression("")))######################Dont ask why
        self._view.b_back.clicked.connect(self._back)
        for index, i in self._view.widget.items():
            i.clicked.connect(self.createitem)
        
    def createitem(self):#,x):
        global df
        widget=self._view.sender()
        #print(df[0][df[0]["id"]==widget.id[0]])
        self.itm=App_item(df[0][df[0]["id"]==widget.id[0]], self._master)
        Control_item(view=self.itm, master=self._master)
        
        self._master.central_widget.addWidget(self.itm)
        self._master.central_widget.setCurrentWidget(self.itm)

    def _back(self):
        print("aaaaaaaa")
        self._master.central_widget.setCurrentWidget(self._master.main_screen)#also found at .widget(0)

In [26]:
class Control_korb:
    def __init__(self, view, master):
        self._view = view
        self._master=master
        self._connectSignalsAndSlots()
        print(self._master.central_widget.count())
        for i in range(1, (self._master.central_widget.count() -1)):
            x=self._master.central_widget.widget(i)
            if x:
                self._master.central_widget.removeWidget(x)
                x.deleteLater()
        
    def _connectSignalsAndSlots(self):
        #self._view.bM.clicked.connect(lambda:(self._buildExpression("")))######################Dont ask why
        self._view.b_abgabe.clicked.connect(lambda:self._checkout(self._view.f_rabatt.text()))
        self._view.b_back.clicked.connect(self._back)
        for id, button in self._view.b_delI.items():
            button.clicked.connect(lambda:self._remove(id))


    def _back(self):
        #print("aaaaaaaa")
        self._master.central_widget.setCurrentWidget(self._master.main_screen)#also found at .widget(0)

    def _remove(self, id):
        global wagen
        global Wahrung
        if wagen[wagen["id"]==id].empty:
            self._view.showError("Irgendwie konnten wir das item nicht finden. melde den Fehler bitte den Admin")
            return
        if wagen[wagen["id"]==id]["amount"].iloc[0]==1:
            wagen.drop(wagen[wagen["id"]==id]["amount"].index, inplace=True)
            self._view.endsum=sum(wagen.amount*wagen.price)
            self._view.l_endsum.setText(str(self._view.endsum)+Wahrung)
            self._view.desc[id].deleteLater()
            self._view.b_delI[id].deleteLater()
            
        elif wagen[wagen["id"]==id]["amount"].iloc[0]>1:
            wagen.loc[wagen["id"]==id, "amount"]-=1
            pds=wagen[wagen["id"]==id].iloc[0]
            self._view.l_amt.setText(str(pds["amount"])+" Stk.="+str(pds["amount"]*pds["price"]))###############################

    def _checkout(self, x):
        global sales
        global wagen
        global df
        if not self._view.endsum:
            return
        if len(x)!=0:
            if x[-1]=="%":
                if can_float(x[1:-1]):
                    x[1:]=self._view.endsum*float("0.x[1:-1]")
                else:
                    self._view.showError("What to do? Use '=', '+' or '-' nad real numbers")
                    return
            if not can_float(x[1:]):
                self._view.showError("What to do? Use '=', '+' or '-' nad real numbers")
                return
            if x[0] in ["+","-"]:
                self._view.endsum=eval("self._view.endsum"+x)
                    
            if x[0]=="=":  
                self._view.endsum=float(x[1:])

        print("still going")
        print(self._view.endsum)
        wagen["time"]=datetime.datetime.now()
        wagen["event"]=[l_Event]*len(wagen)
        wagen["total"]=self._view.endsum
        print(wagen)
        sales=pd.concat([sales,wagen])
        #for i in wagen.iterrows():
        df.loc[df["id"]==wagen["id"], "bestand"]-=wagen["amount"]
        wagen=pd.DataFrame(columns=(df[0].columns.append(pd.Index(["amount"]))))
        safe_verkauf()
        
        self.rueck=App_rueckgeld(self._view.endsum, self._master)
        Control_rueck(view=self.rueck, master=self._master)

        self._master.central_widget.addWidget(self.rueck)
        self._master.central_widget.setCurrentWidget(self.rueck)

####abrechnen

In [27]:
class Control_Hierachie:
    def __init__(self, view, master):
        self._view = view
        self._master=master
        self._connectSignalsAndSlots()

    def _connectSignalsAndSlots(self):
        global df
        self._view.bM.clicked.connect(lambda:(self._buildExpression("")))######################Dont ask why
        self._view.b_back.clicked.connect(self._back)
        for index, i in self._view.hierachie.widget.items():
            i.clicked.connect(self.createitem)

    def createitem(self):#,x):
        global df
        widget=self._view.sender()
        #print(df[0][df[0]["id"]==widget.id[0]])
        self.itm=App_item(df[0][df[0]["id"]==widget.id[0]], self._master)
        Control_item(view=self.itm, master=self._master)
        
        self._master.central_widget.addWidget(self.itm)
        self._master.central_widget.setCurrentWidget(self.itm)

    def _back(self):
        self._master.central_widget.setCurrentWidget(self._master.main_screen)#also found at .widget(0)

In [28]:
class Main_App(QWidget):
    
    def __init__(self, parent=None):
        super(Main_App, self).__init__(parent)
        global df

        #app = QApplication([])
        #settings = App_settings()
        #self._settings=settings
        
        self.layout=QVBoxLayout()
        #centralWidget=QWidget(self)
        self.setLayout(self.layout)
        #self.setCentralWidget(centralWidget)
        self._createOptions()

    def _createOptions(self):
        buttons=QHBoxLayout()
        self.b_settings=QPushButton()
        self.b_settings.setFixedSize(20, 20)
        self.b_korb=QPushButton()
        self.b_korb.setFixedSize(20, 20)
        buttons.addWidget(self.b_settings, alignment=Qt.AlignmentFlag.AlignLeft)
        buttons.addWidget(self.b_korb, alignment=Qt.AlignmentFlag.AlignRight)
        self.layout.addLayout(buttons)

        self.lerrors=QLabel()
        self.layout.addWidget(self.lerrors)

        search=QHBoxLayout()
        self.f_search=QLineEdit()
        self.f_search.setPlaceholderText("suche...")
        self.b_search=QPushButton()
        search.addWidget(self.f_search)
        search.addWidget(self.b_search)
        self.layout.addLayout(search)

        self.b_klasse=QPushButton("Klassen")
        self.layout.addWidget(self.b_klasse)
        
        self.b_QR=QPushButton("QR-Code")
        self.layout.addWidget(self.b_QR)

        self.b_nr=QPushButton("Produktnummer")
        self.layout.addWidget(self.b_nr)

    def showError(self, msg):
        self.lerrors.setText(msg)
        QtTest.QTest.qWait(5000)
        self.lerrors.setText("")

class MainWindow(QMainWindow):

    def __init__(self, parent=None):
        '''
        Constructor
        '''
        QMainWindow.__init__(self, parent)
        self.central_widget=QStackedWidget()
        self.setCentralWidget(self.central_widget)
        self.main_screen = Main_App(self)
        #self.second_screen = Second(self)
        self.central_widget.addWidget(self.main_screen)
        #self.central_widget.addWidget(self.second_screen)
        self.central_widget.setCurrentWidget(self.main_screen)

class Control_I:
    def __init__(self, view, master):
        self._view = view
        self._master=master
        self._connectSignalsAndSlots()

    def showSettings(self):
        self.settings=App_settings(self._master)
        Control_settings(view=self.settings, master=self._master)
        
        self._master.central_widget.addWidget(self.settings)
        self._master.central_widget.setCurrentWidget(self.settings)

    def _connectSignalsAndSlots(self):
        global df
        #self._view.bM.clicked.connect(lambda:(self._buildExpression("")))######################Dont ask why
        self._view.b_search.clicked.connect(partial(self.showSearch, self._view.f_search))
        self._view.f_search.returnPressed.connect(partial(self.showSearch, self._view.f_search))
        self._view.b_settings.clicked.connect(self.showSettings)
        self._view.b_korb.clicked.connect(self.showkorb)
        self._view.b_QR.clicked.connect(self.readqr)
        self._view.b_nr.clicked.connect(self.readnr)
        self._view.b_klasse.clicked.connect(lambda: self.showklasse([]))
        
        

    def showSearch(self, search):
        #app=QApplication([])
        self.search=App_search(search.text(), self._master)
        Control_search(view=self.search, master=self._master)
        
        self._master.central_widget.addWidget(self.search)
        self._master.central_widget.setCurrentWidget(self.search)

    def showkorb(self):
        #app=QApplication([])
        self.korb=App_korb(self._master)
        Control_korb(view=self.korb, master=self._master)

        self._master.central_widget.addWidget(self.korb)
        self._master.central_widget.setCurrentWidget(self.korb)

    def showklasse(self, parent):
        self.hierachie=App_Hierachie(self._master)
        Control_Hierachie(view=self.hierachie, master=self._master)
        
        self._master.central_widget.addWidget(self.hierachie)
        self._master.central_widget.setCurrentWidget(self.hierachie)

    def _back(self):
        self._master.central_widget.setCurrentWidget(self._master.main_screen)#also found at .widget(0)

    def readqr(self):
        global df

        lense=cv2.VideoCapture(0)
        lense.set(5, 640)
        flag=True
        cnter=0
        while flag==True:
            id, frame=lense.read()
            for i in decode(frame):
                x=i.data.decode("utf-8")#######################changed into dictionary?
                id=x[:x.find("\n")].split("    ")
                time.sleep(0.5)
                flag=False
                
            cv2.imshow("something", frame)
            cv2.waitKey(1)
            cnter+=1
            if cnter==31:
                flag=False
        cv2.destroyAllWindows()
        lense.release()
 
        #widget=self._view.sender()
        #print(df[0][df[0]["id"]==widget.id[0]])
        item=df[0][(df[0]["id"]==id[0]) & (df[0]["name"]==id[1])]
        
        if not item.empty:
            self.itm=App_item(item, self._master)
            Control_item(view=self.itm, master=self._master)
            
            self._master.central_widget.addWidget(self.itm)
            self._master.central_widget.setCurrentWidget(self.itm)
        else:
            self._view.showError("Objekt id/Name sind nicht in der Datenbank vorhanden.")
    

    def readnr(self):##############################################################
        global df

        lense=cv2.VideoCapture(0)
        lense.set(5, 640)
        flag=True
        cnter=0
        while flag==True:
            id, frame=lense.read()
            """
            pixel_values=processor(images=frame, return_tensors="pt").pixel_values
            generated_ids=model.generate(pixel_values)
            generated_text=processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
            """
            """
            frame=cv2.convertScaleAbs(frame, alpha=1.25, beta=40)
            norm_img=np.zeros((frame.shape[0], frame.shape[1]))
            frame=cv2.normalize(frame, norm_img, 0, 255, cv2.NORM_MINMAX)
            frame=cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            #frame=cv2.threshold(frame, 0, 200, cv2.THRESH_BINARY)[1]
            frame=cv2.GaussianBlur(frame, (1, 1), 0)
            """
        
            generated_text=pytesseract.image_to_string(frame)
            
            if (generated_text in df[0]["id"]) or cnter==30:
                flag=False
                
            cv2.imshow("something", frame)
            cv2.waitKey(1)
            time.sleep(1)
            cnter+=1
        cv2.destroyAllWindows()
        lense.release()
 
        #print(df[0][df[0]["id"]==widget.id[0]])
        item=df[0][df[0]["id"]==generated_text]
        self.itm=App_item(item, self._master)
        Control_item(view=self.itm, master=self._master)
            
        self._master.central_widget.addWidget(self.itm)
        self._master.central_widget.setCurrentWidget(self.itm)

    
        

def main():
    app=QApplication([])
    #itms=App_item
    window = MainWindow()
    window.show()
    Control_I(view=window.main_screen, master=window)
    sys.exit(app.exec())

if __name__ == "__main__":
    main()
        

SystemExit: 0

C:\Users\Celine\AppData\Local\Programs\Python\Python312\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
